In [ ]:
#default_exp core

# snkrfinder

> API details:
    - sets up the data strucures and tools for Snkr Finder
    
    

### version 0.2 Jan 2021 (0.1 Insight tf/keras)

# OVERVIEW

This is a project initiated while an Insight Data Science fellow.  It grew out of my interest in making data driven tools in the fashion/retail space I had most recently been working.   The original over-scoped idea was to make a shoe desighn tool which could quickly develop some initial sneakers based on choosing some examples, and some text descriptors.  Designs are constrained by the "latent space" defined (discovered?) by a database of shoe images.  However, given the 3 week sprint allowed for development, I pared the tool down to a simple "aesthetic" recommender for sneakers, using the same idea of utilizing an embedding space defined by the database fo shoe images.

In [ ]:
#hide
from nbdev.showdoc import *
import torchvision

## What do we need in this ?




- pretrained architectures
    - mobilenet v2 (from torchvision)
    - resnet (xresnet, from fastai)


- feature extractor
    - headless model with "Identity" output
    
    
- data handling
    - pack into dataframe
    
    - sort files into:
        - train / validate / test?
        - category

    - munging and cleaning and merging
        - zappos 50 k
        - scraped data
            - goat
            - sns
    - datacleaner GUI

- visualization
    - embeddings
        - umap (replace tsne)
    - images 
    

- find "similar" via knn from scikit learn


GUI
    - following the fastai wiget model
    - make some classes 
        - da

EXTENSION
    MODELS
        -categorizers
    AE
    VAE
    CVAE
    
    

In [ ]:
# exports

def get_mnetV2_feature_net(to_cuda=False):
    """
    get mnetv2 from torchvision and put an Identity "head" on it
    """
    ## use an "identity" layer to replace 1000 category classifier
    class Identity(nn.Module):
        def __init__(self):
            super(Identity, self).__init__()

        def forward(self, x):
            return x
    mnetv2  = torchvision.models.mobilenet_v2(pretrained=True)
    mnetv2.classifier = Identity()
    if to_cuda:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device("cpu")
    mnetv2 = mnetv2.to(device)
    mnetv2.eval()

    # just incase we forget the no_grad()
    for param in mnetv2.parameters():
        param.requires_grad = False
        
    return mnetv2

In [ ]:
# export

def munnymunnymunny(makesmoney):
    print("munnnnnnny!")
    profit = f"billz{1000000000}"
    return profit

In [ ]:
# export


DBS = {"zappos": "/ut-zap50k-images",
      "sns": "scraped/sns/im",
      "goat": "scraped/goat/im"}

LATENT_SIZE_SM = 64
LATENT_SIZE_MD = 128
IMG_SIZE_SM = 128
IMG_SIZE_MD = 160
IMG_SIZE = IMG_SIZE_MD
LATENT_SIZE = LATENT_SIZE_MD

ZAPPOS_DF_SIMPLIFIED = "zappos-50k-simplified_sort"

ZAPPOS_FEATS_ALL = "zappos-50k-mobilenetv2-features_"
ZAPPOS_FEATS_ALL_SORT = "zappos-50k-mobilenetv2-features_sort_3"

ZAPPOS_FEATS_MD = f"mobilenetv2-features_medium"

